# HashiCorp Vault Demo for Vault PKI with KV

## Setup of the Demo
<img src="./images/pki-workflow.png" alt="PKI & KV Overview" width="700"/>

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

# Setting up HashiCorp Vault

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
# Expose the Vault API to the host machine.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=root" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-e "VAULT_LOG_LEVEL=DEBUG" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# Vault PKI Secret Engine

<img src="./images/pki-kv-1.png" alt="PKI KV Workflow 1" width="700"/>

## Step 1 - Enable Vault PKI Engine

In [ ]:
# Login to Vault
vault login root

# Enable PKI & KV secrets engine at the default mount path
vault secrets enable pki

## Step 2 - Setting up PKI Secret Engien

In [ ]:
# Create the CA, the private key is kept inside Vault.
# Alternatively, generate an intermediate CA and sign with your root CA.
vault write pki/root/generate/internal \
    common_name=my-org.com \
    ttl=8760h

# configure Vault with the URL's for CRL
vault write pki/config/urls \
    issuing_certificates="http://127.0.0.1:8200/v1/pki/ca" \
    crl_distribution_points="http://127.0.0.1:8200/v1/pki/crl"

In [ ]:
# Configure Roles for Certificate Issue
vault write pki/roles/website1role \
    allowed_domains=website1.com \
    allow_subdomains=true \
    max_ttl=72h

vault write pki/roles/website2role \
    allowed_domains=website2.com \
    allow_subdomains=true \
    max_ttl=72h

## Step 3 - Setting up Access - Approver (ServiceNow) & Requester (Application)

In [ ]:
## Create the Requester Policy with Control Group. Approver is the Security_Team Group.
vault policy write create_pki_cg -<<"EOF"
path "pki/issue/website1role" {
  capabilities = [ "create", "update" ]
 
  control_group = {
    factor "authorizer" {
      identity {
        group_names = [ "security_team" ]
        approvals = 1
      }
    }
  }
}

path "pki/issue/website2role" {
  capabilities = [ "create", "update" ]
 
  control_group = {
    factor "authorizer" {
      identity {
        group_names = [ "security_team" ]
        approvals = 1
      }
    }
  }
}

path "pki/certs" {
  capabilities = [ "read", "list" ]
}

path "pki/cert/*" {
  capabilities = [ "read", "list" ]
}

path "secret/data/*" {
  capabilities = [ "read" ]
}
EOF

In [ ]:
## Create the Approver Policy with Access to Control Group
vault policy write security_team -<<EOF
# To approve the request
path "sys/control-group/authorize" {
    capabilities = ["create", "update"]
}
 
# To check control group request status
path "sys/control-group/request" {
    capabilities = ["create", "update"]
}

path "pki/issue/website1role" {
  capabilities = [ "create", "update" ]
}

path "pki/issue/website2role" {
  capabilities = [ "create", "update" ]
}

path "secret/data/*" {
  capabilities = [ "create", "update", "read" ]
}
EOF

In [ ]:
## Enable AppRole 
vault auth enable approle

## Keep the Mount Accessor
vault auth list -format=json | jq -r '.["approle/"].accessor'  > tmp/approle-accessor.txt

In [ ]:
## Create Role for ServiceNow as part of the Security Team (Approver)
vault write auth/approle/role/approver-role secret_id_ttl=24h token_ttl=20m token_max_ttl=4h
vault read -format=json auth/approle/role/approver-role/role-id | jq -r .data.role_id > tmp/approver-role.txt
vault write -format=json -f auth/approle/role/approver-role/secret-id | jq -r .data.secret_id > tmp/approver-secret.txt

cat tmp/approver-role.txt
cat tmp/approver-secret.txt

vault write -format=json identity/entity name="Service Now" \
        metadata=team="Controller"

vault read -format=json identity/entity/name/"Service Now" | jq -r ".data.id" > tmp/entity_id_servicenow.txt

vault write identity/entity-alias name=$(cat tmp/approver-role.txt) \
      canonical_id=$(cat tmp/entity_id_servicenow.txt) \
      mount_accessor=$(cat tmp/approle-accessor.txt)

## Add ServiceNow To Security_Team Group
vault write identity/group name="security_team" \
      policies="security_team" \
      member_entity_ids=$(cat tmp/entity_id_servicenow.txt)



In [ ]:
## Create Application Role as part of the Developer Team (Requester)
vault write auth/approle/role/app-role token_policies="create_pki_cg" secret_id_ttl=24h token_ttl=20m token_max_ttl=4h
vault read -format=json auth/approle/role/app-role/role-id | jq -r .data.role_id > tmp/app-role.txt
vault write -format=json -f auth/approle/role/app-role/secret-id | jq -r .data.secret_id > tmp/app-secret.txt

cat tmp/app-role.txt
cat tmp/app-secret.txt

## Step 4 - Create Certificate & Transfer to KV (ServiceNow)
<img src="./images/pki-kv-2.png" alt="PKI KV Workflow 1" width="700"/>

In [ ]:
## Simulate ServiceNow Login via AppRole
 
vault write -format=json auth/approle/login \
    role_id=$(cat tmp/approver-role.txt) \
    secret_id=$(cat tmp/approver-secret.txt) | jq -r .auth.client_token > tmp/servicenow-token.txt

vault login $(cat tmp/servicenow-token.txt)

In [ ]:
# ServiceNow Request for the initial certificate
mkdir -p tmp/initial_cert

vault write pki/issue/website1role common_name=www.website1.com ttl=72h -format=json > tmp/initial_cert/certificate.txt

## Extract the certificate
cat tmp/initial_cert/certificate.txt | jq -r ".data.certificate" > tmp/initial_cert/certificate.pem
cat tmp/initial_cert/certificate.txt | jq -r ".data.private_key" > tmp/initial_cert/private_key.pem
cat tmp/initial_cert/certificate.pem 
cat tmp/initial_cert/private_key.pem

In [ ]:
# ServiceNow Loaded the Certificate into KV Store
vault kv put secret/application1 \
    cert=@tmp/initial_cert/certificate.pem \
    key=@tmp/initial_cert/private_key.pem

In [ ]:
vault kv get secret/application1

## Step 5 - Configure App or Deploy Vault Agent

<img src="./images/pki-kv-3.png" alt="PKI KV 3" width="700"/>

In [ ]:
# Vault Agent Config
tee tmp/agent-config.hcl <<EOF
pid_file = "./pidfile"

vault {
  address = "$VAULT_ADDR"
  retry {
    num_retries = 5
  }
}

auto_auth {
  method {
    type = "approle"
    config = {
      role_id_file_path = "tmp/app-role.txt"
      secret_id_file_path = "tmp/app-secret.txt"
      remove_secret_id_file_after_reading = false
    }
  }

  sink "file" {
    config = {
      path = "tmp/token"
    }
  }
}

cache {
}

template_config {
  exit_on_retry_failure = true
  static_secret_render_interval = "30s"
}

template {
  contents = "{{with secret \"secret/application1\" }}{{.Data.data.cert}}{{ end }}"
  destination = "tmp/agent/cert.crt"
}

template {
  contents = "{{with secret \"secret/application1\"}}{{.Data.data.key}}{{ end }}"
  destination = "tmp/agent/key.pem"
}

EOF

In [ ]:
# Run Vault Agent
vault agent -log-level debug -config=tmp/agent-config.hcl

In [ ]:
# Check Cert Rendered by Vault Agent
openssl x509 -in tmp/agent/cert.crt -text -noout | grep Subject: 

## Step 6 - Cron Job - Rotation (Simulate Approval with Control Group instead of ServiceNow)

<img src="./images/pki-kv-4.png" alt="PKI KV 4" width="700"/>

In [ ]:
## Simulate App Login via AppRole
 
vault write -format=json auth/approle/login \
    role_id=$(cat tmp/app-role.txt) \
    secret_id=$(cat tmp/app-secret.txt) | jq -r .auth.client_token > tmp/app-token.txt

vault login $(cat tmp/app-token.txt)

In [ ]:
# Example Script to Capture Expiring Certificate
./get_vault_pki_certs.sh -a $VAULT_ADDR -t $(cat tmp/app-token.txt) -d 90

In [ ]:
# Submit Cert Rotation Request with Control Group

mkdir -p tmp/controlgroup

vault write pki/issue/website2role common_name=www.website2.com ttl=72h -format=json > tmp/controlgroup/wrapped_response.txt

## Extract and store the token & accessor
cat tmp/controlgroup/wrapped_response.txt
cat tmp/controlgroup/wrapped_response.txt | jq -r ".wrap_info.token" > tmp/controlgroup/wrapped_token.txt
cat tmp/controlgroup/wrapped_response.txt | jq -r ".wrap_info.accessor" > tmp/controlgroup/wrapped_accessor.txt


## Step 6 - ServiceNow to Approve Control Group and Update KV

<img src="./images/pki-kv-5.png" alt="PKI KV 4" width="700"/>

In [ ]:
## Simulate ServiceNow Login via AppRole To Approve
 
vault write -format=json auth/approle/login \
    role_id=$(cat tmp/approver-role.txt) \
    secret_id=$(cat tmp/approver-secret.txt) | jq -r .auth.client_token > tmp/servicenow-token.txt

vault login $(cat tmp/servicenow-token.txt)

In [ ]:
# Approve the Request
vault write sys/control-group/authorize accessor=$(cat tmp/controlgroup/wrapped_accessor.txt)

In [ ]:
# ServiceNow Request for the rotated certificate
mkdir -p tmp/rotated_cert

vault unwrap -format=json $(cat tmp/controlgroup/wrapped_token.txt) > tmp/rotated_cert/certificate.txt

## Extract the certificate
cat tmp/rotated_cert/certificate.txt | jq -r ".data.certificate" > tmp/rotated_cert/certificate.pem
cat tmp/rotated_cert/certificate.txt | jq -r ".data.private_key" > tmp/rotated_cert/private_key.pem
cat tmp/rotated_cert/certificate.pem 
cat tmp/rotated_cert/private_key.pem

In [ ]:
# ServiceNow Loaded the Certificate into KV Store
vault kv put secret/application1 \
    cert=@tmp/rotated_cert/certificate.pem \
    key=@tmp/rotated_cert/private_key.pem

In [ ]:
openssl x509 -in tmp/agent/cert.crt -text -noout | grep Subject:

# Recap
<img src="./images/pki-workflow.png" alt="PKI & KV Overview" width="700"/>

# Cleanup

In [1]:
# Cleanup
vault login root

# Disable PKI secrets engine
vault secrets disable pki

# Disable Userpass
vault auth disable approle

# Remove all the tmp file
rm -rf tmp/*

WARNING! VAULT_ADDR and -address unset. Defaulting to https://127.0.0.1:8200.
Error authenticating: error looking up token: Get "https://127.0.0.1:8200/v1/auth/token/lookup-self": http: server gave HTTP response to HTTPS client
WARNING! VAULT_ADDR and -address unset. Defaulting to https://127.0.0.1:8200.
Error disabling secrets engine at pki/: Delete "https://127.0.0.1:8200/v1/sys/mounts/pki": http: server gave HTTP response to HTTPS client
WARNING! VAULT_ADDR and -address unset. Defaulting to https://127.0.0.1:8200.
Error disabling auth method at approle/: Delete "https://127.0.0.1:8200/v1/sys/auth/approle": http: server gave HTTP response to HTTPS client


In [2]:
# Stop Vault container
docker stop vault-enterprise

vault-enterprise
